In [49]:
#Siamese Architecture_Simple_LSTM
#keras_1.1.2_theano_backend

In [1]:
import os

with open(os.path.expanduser('~')+'/.keras/keras.json','w') as f:
    new_settings = """{\r\n
    "epsilon": 1e-07,\r\n
    "image_data_format": "channels_last",\n
    "backend": "theano",\r\n
    "floatx": "float32"\r\n
    }"""
    f.write(new_settings)

import keras

Using Theano backend.


In [2]:
import pandas as pd
import nltk
import numpy as np

In [3]:
#WordVectors

In [4]:
import gensim
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [5]:
train_DF=pd.read_csv('train.csv',index_col='id')

In [6]:
train_DF.reset_index(drop=True,inplace=True)
train_DF['question1'] = train_DF['question1'].astype(str)
train_DF['question2'] = train_DF['question2'].astype(str)

In [27]:
import re
numbers=re.compile(r'(\d+\.\d+|\d+)')
def cleanText(text):
    return numbers.sub('N',text)

In [28]:
texts = []
questions1=[]
questions2=[]
duplicates=[]

for idx in range(train_DF.question1.shape[0]):
    text=train_DF.question1[idx]
    text=cleanText(text)
    texts.append(text)
    words = text_to_word_sequence(text)
    questions1.append(words)
    
    text=train_DF.question2[idx]
    text=cleanText(text)
    texts.append(text)
    words = text_to_word_sequence(text)
    questions2.append(words)
    
    duplicates.append(train_DF.is_duplicate[idx])

In [29]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten,merge
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

In [47]:
MAX_NB_WORDS = 40000
VALIDATION_SPLIT = 0.2

In [30]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 90278 unique tokens.


In [31]:
vocab_dim = 300 # dimensionality of your word vectors
n_symbols = len(word_index) + 1 # adding 1 to account for 0th index (for masking)
embedding_weights = np.random.random((n_symbols,vocab_dim))
for word,index in word_index.items():
    if word in model.wv.vocab:
        embedding_weights[index] = model.word_vec(word)       

In [32]:
embedding_weights.shape

(90279, 300)

In [33]:
MAX_SENT_LENGTH = 250

In [34]:
questions1[1]

['what', 'is', 'the', 'story', 'of', 'kohinoor', 'koh', 'i', 'noor', 'diamond']

In [35]:
data_question1 = np.zeros((len(questions1), MAX_SENT_LENGTH), dtype='int32')
for i, sentence in enumerate(questions1):
    for j, word in enumerate(sentence):
        if j<=MAX_SENT_LENGTH:
            data_question1[i,j] = tokenizer.word_index[word]

In [36]:
data_question2 = np.zeros((len(questions2), MAX_SENT_LENGTH), dtype='int32')
for i, sentence in enumerate(questions2):
    for j, word in enumerate(sentence):
        if j<=MAX_SENT_LENGTH:
            data_question2[i,j] = tokenizer.word_index[word]

In [37]:
embedding_layer=Embedding(input_dim=len(word_index)+1,output_dim=vocab_dim,weights=[embedding_weights],
                          input_length=MAX_SENT_LENGTH,trainable=True)

In [38]:
data_question1_1000_train=data_question1[1000:2000]
data_question2_1000_train=data_question1[1000:2000]
duplicates_1000_train=duplicates[1000:2000]

In [39]:
data_question1_1000_vali=data_question1[2000:3000]
data_question2_1000_vali=data_question1[2000:3000]
duplicates_1000_vali=duplicates[2000:3000]

In [40]:
data_question1_1000_test=data_question1[:1000]
data_question2_1000_test=data_question1[:1000]
duplicates_1000_test=duplicates[:1000]

In [41]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [44]:
#Input for each Question
left_question_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
right_question_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')

#Encoders at Question Level
embedded_sequences = embedding_layer(left_question_input)
left_lstm_sent = Bidirectional(LSTM(300))(embedded_sequences)

embedded_sequences = embedding_layer(right_question_input)
right_lstm_sent = Bidirectional(LSTM(300))(embedded_sequences)

#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([left_lstm_sent,right_lstm_sent], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(1,activation='sigmoid')(both)

siamese_net = Model(input=[left_question_input,right_question_input],output=prediction)
optimizer = Adam(0.00006)
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

siamese_net.count_params()

print("model - Siamese Networks")
print(siamese_net.summary())

model - Siamese Networks
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 250)           0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 250)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 250, 300)      27083700    input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 600)           1442400    

In [45]:
#data_question2_1000_train.shape
duplicates_1000_train=np.array(duplicates_1000_train).reshape(1000,1)
duplicates_1000_vali=np.array(duplicates_1000_vali).reshape(1000,1)

In [46]:
siamese_net.fit([data_question1_1000_train,data_question2_1000_train], duplicates_1000_train,
                validation_data=([data_question1_1000_vali,data_question2_1000_vali],duplicates_1000_vali)
                                  ,nb_epoch=10, batch_size=50)

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 122s - loss: 0.6532 - val_loss: 0.6711
Epoch 2/10
1000/1000 [==============================] - 123s - loss: 0.6323 - val_loss: 0.6615
Epoch 3/10
1000/1000 [==============================] - 121s - loss: 0.6236 - val_loss: 0.6573
Epoch 4/10
1000/1000 [==============================] - 122s - loss: 0.6206 - val_loss: 0.6562
Epoch 5/10
1000/1000 [==============================] - 122s - loss: 0.6178 - val_loss: 0.6571
Epoch 6/10
1000/1000 [==============================] - 121s - loss: 0.6108 - val_loss: 0.6557
Epoch 7/10
1000/1000 [==============================] - 122s - loss: 0.6082 - val_loss: 0.6694
Epoch 8/10
1000/1000 [==============================] - 127s - loss: 0.6166 - val_loss: 0.6508
Epoch 9/10
1000/1000 [==============================] - 132s - loss: 0.6098 - val_loss: 0.6594
Epoch 10/10
1000/1000 [==============================] - 143s - loss: 0.6026 - val_loss: 0.6543
